In [1]:
import numpy as np 
import pandas as pd
import os
import gc
import re

# feature libraries
from scipy.sparse import hstack, csr_matrix
from sklearn import preprocessing

In [2]:
data_dir = '../input/kaggle_data'
feat_dir = '../input/features'

In [5]:
# used_cols = ['item_id','image']

# train = pd.read_csv(f'{data_dir}/train.csv', index_col="item_id", usecols=used_cols)
# test = pd.read_csv(f'{data_dir}/test.csv', index_col="item_id", usecols=used_cols)

# train_idx = train.index
# test_idx = test.index

# df = pd.concat([train, test], axis=0)

# del train, test
# gc.collect()

14

In [33]:
img_cols = ['img_size_x', 'img_size_y', 'img_file_size', 'img_mean_color', 'img_dullness_light_percent', 
 'img_dullness_dark_percent', 'img_blur', 'img_blue_mean', 'img_green_mean', 'img_red_mean',
 'img_blue_std', 'img_green_std', 'img_red_std', 'img_average_red', 'img_average_green',
 'img_average_blue', 'img_sobel00', 'img_sobel10', 'img_sobel20', 'img_sobel01', 'img_sobel11',
 'img_sobel21', 'img_kurtosis', 'img_skew', 'thing1', 'thing2']

train_img = pd.read_csv(f'{feat_dir}/train_img_data.csv', index_col="item_id", usecols=img_cols+['item_id'])
test_img = pd.read_csv(f'{feat_dir}/test_img_data.csv', index_col="item_id", usecols=img_cols+['item_id'])

train_idx = train_img.index
test_idx = test_img.index

df = pd.concat([train_img, test_img], axis=0)
del train_img, test_img

In [34]:
for c in img_cols:
    df[c].fillna(value=0, inplace=True)
    df[c] = (df[c] - np.mean(df[c]))/np.std(df[c])

In [35]:
df.head()

,img_size_x,img_size_y,img_file_size,img_mean_color,thing1,thing2,img_sobel00,img_sobel10,img_sobel20,img_sobel01,...,img_blur,img_blue_mean,img_green_mean,img_red_mean,img_blue_std,img_green_std,img_red_std,img_average_red,img_average_green,img_average_blue
item_id,,,,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,-0.151260,0.759351,-0.498618,-0.067603,-0.144588,-0.157748,0.262605,0.262605,0.262605,0.183615,...,-0.345490,-0.280614,-0.638004,0.687033,0.641418,0.851439,0.132975,-0.280614,-0.638004,0.687033
2dac0150717d,-0.138074,0.759351,-0.318181,-0.435307,-0.144422,-0.156697,0.231931,0.231931,0.231931,0.156987,...,0.275917,-0.608886,-0.394327,-0.266605,0.451465,0.240082,0.133307,-0.608886,-0.394327,-0.266605
ba83aefab5dc,0.072903,-0.178130,-0.949332,1.376228,-0.126514,-0.075738,-0.179758,-0.179758,-0.179758,0.218111,...,-0.254716,1.487169,1.379419,1.140900,1.906005,1.904254,1.820077,1.487169,1.379419,1.140900
02996f1dd2ea,-0.138074,-0.178130,-1.220311,2.143408,-0.144422,-0.157223,-1.143822,-1.143822,-1.143822,-1.089559,...,-0.372882,2.253272,2.131517,1.853883,0.683302,0.762878,0.720346,2.253272,2.131517,1.853883
7c90be56d2ab,1.707973,-0.178130,0.022901,0.204425,-0.142598,-0.149863,-0.202544,-0.202544,-0.202544,0.332612,...,-0.189964,0.175604,0.376266,0.050487,0.446751,0.359739,0.191530,0.175604,0.376266,0.050487


In [36]:
df.columns[df.isna().any()].tolist()

[]

In [37]:
feat_cols = list(df.columns)[0:]
feat_cols

['img_size_x',
 'img_size_y',
 'img_file_size',
 'img_mean_color',
 'thing1',
 'thing2',
 'img_sobel00',
 'img_sobel10',
 'img_sobel20',
 'img_sobel01',
 'img_sobel11',
 'img_sobel21',
 'img_kurtosis',
 'img_skew',
 'img_dullness_light_percent',
 'img_dullness_dark_percent',
 'img_blur',
 'img_blue_mean',
 'img_green_mean',
 'img_red_mean',
 'img_blue_std',
 'img_green_std',
 'img_red_std',
 'img_average_red',
 'img_average_green',
 'img_average_blue']

In [38]:
train_img = df.loc[train_idx,:]
train_img.head()

,img_size_x,img_size_y,img_file_size,img_mean_color,thing1,thing2,img_sobel00,img_sobel10,img_sobel20,img_sobel01,...,img_blur,img_blue_mean,img_green_mean,img_red_mean,img_blue_std,img_green_std,img_red_std,img_average_red,img_average_green,img_average_blue
item_id,,,,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,-0.151260,0.759351,-0.498618,-0.067603,-0.144588,-0.157748,0.262605,0.262605,0.262605,0.183615,...,-0.345490,-0.280614,-0.638004,0.687033,0.641418,0.851439,0.132975,-0.280614,-0.638004,0.687033
2dac0150717d,-0.138074,0.759351,-0.318181,-0.435307,-0.144422,-0.156697,0.231931,0.231931,0.231931,0.156987,...,0.275917,-0.608886,-0.394327,-0.266605,0.451465,0.240082,0.133307,-0.608886,-0.394327,-0.266605
ba83aefab5dc,0.072903,-0.178130,-0.949332,1.376228,-0.126514,-0.075738,-0.179758,-0.179758,-0.179758,0.218111,...,-0.254716,1.487169,1.379419,1.140900,1.906005,1.904254,1.820077,1.487169,1.379419,1.140900
02996f1dd2ea,-0.138074,-0.178130,-1.220311,2.143408,-0.144422,-0.157223,-1.143822,-1.143822,-1.143822,-1.089559,...,-0.372882,2.253272,2.131517,1.853883,0.683302,0.762878,0.720346,2.253272,2.131517,1.853883
7c90be56d2ab,1.707973,-0.178130,0.022901,0.204425,-0.142598,-0.149863,-0.202544,-0.202544,-0.202544,0.332612,...,-0.189964,0.175604,0.376266,0.050487,0.446751,0.359739,0.191530,0.175604,0.376266,0.050487


In [39]:
test_img = df.loc[test_idx,:]
test_img.head()

,img_size_x,img_size_y,img_file_size,img_mean_color,thing1,thing2,img_sobel00,img_sobel10,img_sobel20,img_sobel01,...,img_blur,img_blue_mean,img_green_mean,img_red_mean,img_blue_std,img_green_std,img_red_std,img_average_red,img_average_green,img_average_blue
item_id,,,,,,,,,,,,,,,,,,,,,
6544e41a8817,0.653089,-0.17813,0.282286,-0.157379,-0.143925,-0.156697,0.270893,0.270893,0.270893,0.325967,...,-0.437174,-0.034905,-0.123148,-0.294301,1.122684,1.272147,1.246686,-0.034905,-0.123148,-0.294301
65b9484d670f,-2.511564,-2.99057,-1.956726,-2.356342,-0.144588,-0.157748,-3.082198,-3.082198,-3.082198,-3.091064,...,-0.755356,-2.144440,-2.323855,-2.378830,-2.455520,-2.472494,-2.398724,-2.144440,-2.323855,-2.378830
8bab230b2ecd,1.707973,-0.17813,1.381192,-0.727184,1.608395,5.774847,0.250045,0.250045,0.250045,0.495709,...,0.016780,-0.383574,-0.730401,-0.991091,1.414435,1.143024,0.846600,-0.383574,-0.730401,-0.991091
8e348601fefc,-2.511564,-2.99057,-1.956726,-2.356342,-0.144588,-0.157748,-3.082198,-3.082198,-3.082198,-3.091064,...,-0.755356,-2.144440,-2.323855,-2.378830,-2.455520,-2.472494,-2.398724,-2.144440,-2.323855,-2.378830
8bd2fe400b89,0.653089,-0.17813,0.711753,-0.821029,-0.143261,-0.150914,-0.072423,-0.072423,-0.072423,0.037342,...,-0.405786,-0.937085,-0.886353,-0.572041,-0.070493,-0.398373,-0.172924,-0.937085,-0.886353,-0.572041


In [40]:
train_img.to_csv(f'{feat_dir}/train_img.csv', index=True, header=True)
test_img.to_csv(f'{feat_dir}/test_img.csv', index=True, header=True)
